In [37]:
from neo.rawio import SpikeGadgetsRawIO

In [38]:
rec_file_path = "/Users/rly/Downloads/20230622_155936/20230622_155936.rec"

In [39]:
neo_io = SpikeGadgetsRawIO(filename=rec_file_path)  # all streams

In [67]:
neo_io.parse_header()
header = neo_io.header

In [41]:
from xml.etree import ElementTree

# parse file until "</Configuration>"
header_size = None
with open(rec_file_path, mode='rb') as f:
    while True:
        line = f.readline()
        if b"</Configuration>" in line:
            header_size = f.tell()
            break

    if header_size is None:
        ValueError("SpikeGadgets: the xml header does not contain '</Configuration>'")

    f.seek(0)
    header_txt = f.read(header_size).decode('utf8')

# explore xml header
root = ElementTree.fromstring(header_txt)
gconf = sr = root.find('GlobalConfiguration')
hconf = root.find('HardwareConfiguration')
sconf = root.find('SpikeConfiguration')

_sampling_rate = float(hconf.attrib['samplingRate'])
num_ephy_channels = int(hconf.attrib['numChannels'])

In [42]:
print(f'There are {len(hconf)} devices.')

There are 4 devices.


In [43]:
# explore sub stream and count packet size

# The raw data block consists of N packets.
# Each packet consists of:
# First byte is 0x55
# Some number of bytes for each device (e.g., Controller_DIO has 1 byte, 
# ECU has 32 bytes, Multiplexed has 8 bytes, SysClock has 8 bytes)
# Timestamp (uint32) which has 4 bytes
# Ephys data (int16) which has 2 * num_ephy_channels bytes

# first byte is 0x55
packet_size = 1

# save the number of bytes for each device and update packet_size
stream_bytes = {}
for device in hconf:
    stream_id = device.attrib['name']
    num_bytes = int(device.attrib['numBytes'])
    stream_bytes[stream_id] = packet_size
    packet_size += num_bytes
    print(f"Device '{stream_id}' starts at byte index {stream_bytes[stream_id]} and has {num_bytes} bytes.")
print(packet_size)
print(stream_bytes)  # this is the byte index for each stream

Device 'Controller_DIO' starts at byte index 1 and has 1 bytes.
Device 'ECU' starts at byte index 2 and has 32 bytes.
Device 'Multiplexed' starts at byte index 34 and has 8 bytes.
Device 'SysClock' starts at byte index 42 and has 8 bytes.
50
{'Controller_DIO': 1, 'ECU': 2, 'Multiplexed': 34, 'SysClock': 42}


In [44]:
# timestamps 4 uint32
_timestamp_byte = packet_size  # store the current packet size as the index of the timestamp byte
packet_size += 4  # the timestamp is uint32, which is 4 bytes
print(f"The timestamp (uint32) starts at byte {_timestamp_byte} and has 4 bytes.")
print(f"The ephys data (int16) starts at byte {packet_size} and has {2 * num_ephy_channels} bytes (2 bytes for each of the {num_ephy_channels} channels).")

packet_size += 2 * num_ephy_channels
print(f"The total packet size is {packet_size} bytes.")

The timestamp (uint32) starts at byte 50 and has 4 bytes.
The ephys data (int16) starts at byte 54 and has 256 bytes (2 bytes for each of the 128 channels).
The total packet size is 310 bytes.


In [45]:
# read the binary part lazily
import numpy as np
raw_memmap = np.memmap(rec_file_path, mode='r', offset=header_size, dtype='<u1')

num_packet = raw_memmap.size // packet_size
raw_memmap = raw_memmap[:num_packet * packet_size]
_raw_memmap = raw_memmap.reshape(-1, packet_size)  # reshape to (num_packets, packet_size)
print(_raw_memmap.shape)

(314266, 310)


In [46]:
# create signal channels
stream_ids = []
signal_streams = []
signal_channels = []

In [21]:
# walk in xml device and keep only "analog" data types
# only the Analog ECU data gets processed
_mask_channels_bytes = {}
for device in hconf:
    stream_id = device.attrib['name']
    print(stream_id)
    
    for channel in device:
        print(channel.attrib)
        
        if 'interleavedDataIDByte' in channel.attrib:
            # TODO LATER: deal with "headstageSensor" which have interleaved
            continue

        if channel.attrib['dataType'] == 'analog':

            # add to stream_ids, signal_streams, and _mask_channels_bytes
            # only if there are channels to read
            if stream_id not in stream_ids:
                stream_ids.append(stream_id)
                stream_name = stream_id
                signal_streams.append((stream_name, stream_id))
                _mask_channels_bytes[stream_id] = []

            name = channel.attrib['id']
            chan_id = channel.attrib['id']
            dtype = 'int16'
            # TODO LATER : handle gain correctly according the file version
            units = ''
            gain = 1.
            offset = 0.

            # xml header has, for each ntrode,
            # rawScalingToUv, spikeScalingToUv, and lfpScalingToUv
            signal_channels.append((name, chan_id, _sampling_rate, 'int16',
                                 units, gain, offset, stream_id))

            # to handle digital data, need to split the data by bits
            num_bytes = stream_bytes[stream_id] + int(channel.attrib['startByte'])
            chan_mask = np.zeros(packet_size, dtype='bool')
            chan_mask[num_bytes] = True
            chan_mask[num_bytes + 1] = True
            _mask_channels_bytes[stream_id].append(chan_mask)


Controller_DIO
{'dataType': 'digital', 'bit': '0', 'startByte': '0', 'id': 'Controller_Din1', 'input': '1'}
{'dataType': 'digital', 'bit': '1', 'startByte': '0', 'id': 'Controller_Din2', 'input': '1'}
{'dataType': 'digital', 'bit': '2', 'startByte': '0', 'id': 'Controller_Din3', 'input': '1'}
{'dataType': 'digital', 'bit': '3', 'startByte': '0', 'id': 'Controller_Din4', 'input': '1'}
{'dataType': 'digital', 'bit': '4', 'startByte': '0', 'id': 'Controller_Din5', 'input': '1'}
{'dataType': 'digital', 'bit': '5', 'startByte': '0', 'id': 'Controller_Din6', 'input': '1'}
{'dataType': 'digital', 'bit': '6', 'startByte': '0', 'id': 'Controller_Din7', 'input': '1'}
{'dataType': 'digital', 'bit': '7', 'startByte': '0', 'id': 'Controller_Din8', 'input': '1'}
ECU
{'dataType': 'digital', 'bit': '0', 'startByte': '0', 'id': 'ECU_Din1', 'input': '1'}
{'dataType': 'digital', 'bit': '1', 'startByte': '0', 'id': 'ECU_Din2', 'input': '1'}
{'dataType': 'digital', 'bit': '2', 'startByte': '0', 'id': 'ECU_

In [48]:
if num_ephy_channels > 0:
    stream_id = 'trodes'
    stream_name = stream_id
    signal_streams.append((stream_name, stream_id))
    _mask_channels_bytes[stream_id] = []

    chan_ind = 0
    for trode in sconf:
        for schan in trode:
            name = 'trode' + trode.attrib['id'] + 'chan' + schan.attrib['hwChan']
            chan_id = schan.attrib['hwChan']
            # TODO LATER : handle gain correctly according the file version
            units = ''
            gain = 1.
            offset = 0.
            signal_channels.append((name, chan_id, _sampling_rate, 'int16',
                                 units, gain, offset, stream_id))

            chan_mask = np.zeros(packet_size, dtype='bool')
            # use the channel index (0 to N-1, in order) to compute the starting index of the
            # two bytes in each packet that corresponds to int16 data for this
            # ephys channel. then set the mask for those two bytes to Triue
            num_bytes = packet_size - 2 * num_ephy_channels + 2 * chan_ind
            chan_mask[num_bytes] = True
            chan_mask[num_bytes + 1] = True
            _mask_channels_bytes[stream_id].append(chan_mask)

            chan_ind += 1


In [50]:
# _mask_channels_bytes is now a dictionary mapping stream name ('ECU', 'trodes') to a list of masks
# _mask_channels_bytes

In [54]:
_signal_stream_dtype = [
    ('name', 'U64'),  # not necessarily unique
    ('id', 'U64'),  # must be unique
]

_signal_channel_dtype = [
    ('name', 'U64'),  # not necessarily unique
    ('id', 'U64'),  # must be unique
    ('sampling_rate', 'float64'),
    ('dtype', 'U16'),
    ('units', 'U64'),
    ('gain', 'float64'),
    ('offset', 'float64'),
    ('stream_id', 'U64'),
]

In [57]:
selected_streams = None

In [58]:
# make mask as array (used in _get_analogsignal_chunk(...))
_mask_streams = {}
for stream_id, l in _mask_channels_bytes.items():
    mask = np.array(l)
    _mask_channels_bytes[stream_id] = mask
    _mask_streams[stream_id] = np.any(mask, axis=0)

signal_streams = np.array(signal_streams, dtype=_signal_stream_dtype)
signal_channels = np.array(signal_channels, dtype=_signal_channel_dtype)

# remove some stream if no wanted
if selected_streams is not None:
    if isinstance(selected_streams, str):
        selected_streams = [selected_streams]
    assert isinstance(selected_streams, list)

    keep = np.in1d(signal_streams['id'], selected_streams)
    signal_streams = signal_streams[keep]

    keep = np.in1d(signal_channels['stream_id'], selected_streams)
    signal_channels = signal_channels[keep]

In [62]:
print(type(_mask_channels_bytes["ECU"]))
print(_mask_channels_bytes["ECU"].shape)

<class 'numpy.ndarray'>
(12, 310)


In [64]:
print(type(_mask_streams["ECU"]))
print(_mask_streams["ECU"].shape)
_mask_streams["ECU"]

<class 'numpy.ndarray'>
(310,)


array([False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [65]:
gconf.attrib

{'saveDisplayedChanOnly': '1',
 'filePath': '',
 'headstageFirmwareVersion': '4.4',
 'headstageSmartRefOn': '0',
 'headstageGyroSensorOn': '1',
 'headstageSerial': '01504 00126',
 'headstageAutoSettleOn': '0',
 'compileDate': 'May 24 2023',
 'trodesVersion': '2.4.0',
 'controllerFirmwareVersion': '3.18',
 'headstageAccelSensorOn': '1',
 'commitHead': 'heads/Release_2.4.0-0-g499429f3',
 'controllerSerial': '65535 65535',
 'timestampAtCreation': '51493215',
 'systemTimeAtCreation': '       1687474797888',
 'headstageMagSensorOn': '0',
 'filePrefix': '',
 'compileTime': '10:59:15',
 'qtVersion': '6.2.2',
 'realtimeMode': '0'}

In [69]:
block_index = 0
seg_index = 0
i_start = 0
i_stop = 10
stream_index = 0
channel_indexes = [0, 1]

In [71]:
_raw_memmap.shape

(314266, 310)

In [73]:
stream_id = header['signal_streams'][stream_index]['id']

raw_unit8 = _raw_memmap[i_start:i_stop]  # this should say uint8
raw_unit8.shape

(10, 310)

In [80]:
_raw_memmap.dtype

dtype('uint8')

In [75]:
np.arange(num_chan)[slice(0, 5)]

array([0, 1, 2, 3, 4])

In [ ]:
num_chan = len(_mask_channels_bytes[stream_id])
re_order = None
if channel_indexes is None:
    # no loop : entire stream mask
    stream_mask = _mask_streams[stream_id]
else:
    # accumulate mask
    if isinstance(channel_indexes, slice):
        chan_inds = np.arange(num_chan)[channel_indexes]  # this seems redundant...
    else:
        chan_inds = channel_indexes

        if np.any(np.diff(channel_indexes) < 0):
            # handle channel are not ordered
            sorted_channel_indexes = np.sort(channel_indexes)
            re_order = np.array([list(sorted_channel_indexes).index(ch)
                                 for ch in channel_indexes])

    stream_mask = np.zeros(raw_unit8.shape[1], dtype='bool')
    for chan_ind in chan_inds:
        chan_mask = _mask_channels_bytes[stream_id][chan_ind]
        stream_mask |= chan_mask

# this copies the data from the memmap into memory
raw_unit8_mask = raw_unit8[:, stream_mask]

# the data are int16
shape = raw_unit8_mask.shape
shape = (shape[0], shape[1] // 2)
# reshape the and retype by view
raw_unit16 = raw_unit8_mask.flatten().view('int16').reshape(shape)  # this should be int16

if re_order is not None:
    raw_unit16 = raw_unit16[:, re_order]

In [79]:
raw_unit8_mask.shape

(10, 4)

In [81]:
shape

(10, 2)

In [83]:
raw_unit16.shape

(10, 2)

In [86]:
raw_unit8_mask

array([[ 13, 223,   5, 222],
       [  5, 223,   5, 222],
       [ 13, 223,   5, 222],
       [ 13, 223, 253, 221],
       [  5, 223,   5, 222],
       [253, 222, 253, 221],
       [ 13, 223,   5, 222],
       [ 13, 223, 253, 221],
       [  5, 223,   5, 222],
       [ 13, 223,   5, 222]], dtype=uint8)

In [87]:
raw_unit16

array([[-8435, -8699],
       [-8443, -8699],
       [-8435, -8699],
       [-8435, -8707],
       [-8443, -8699],
       [-8451, -8707],
       [-8435, -8699],
       [-8435, -8707],
       [-8443, -8699],
       [-8435, -8699]], dtype=int16)